# Modelling a gallium arsenide surface

This example shows how to use the atomistic simulation environment or ASE for short,
to set up and run a particular calculation of a gallium arsenide surface.
ASE is a Python package to simplify the process of setting up,
running and analysing results from atomistic simulations across different simulation codes.
For more details on the integration DFTK provides with ASE,
see Atomistic simulation environment.

In this example we will consider modelling the (1, 1, 0) GaAs surface separated by vacuum.

Parameters of the calculation. Since this surface is far from easy to converge,
we made the problem simpler by choosing a smaller `Ecut` and smaller values
for `n_GaAs` and `n_vacuum`.
More interesting settings are `Ecut = 15` and `n_GaAs = n_vacuum = 20`.

In [1]:
miller = (1, 1, 0)   # Surface Miller indices
n_GaAs = 2           # Number of GaAs layers
n_vacuum = 4         # Number of vacuum layers
Ecut = 5             # Hartree
kgrid = (4, 4, 1);   # Monkhorst-Pack mesh

Use ASE to build the structure:

In [2]:
using ASEconvert
using PythonCall

a = 5.6537  # GaAs lattice parameter in Ångström (because ASE uses Å as length unit)
gaas = ase.build.bulk("GaAs", "zincblende"; a)
surface = ase.build.surface(gaas, miller, n_GaAs, 0, periodic=true);

Get the amount of vacuum in Ångström we need to add

In [3]:
d_vacuum = maximum(maximum, surface.cell) / n_GaAs * n_vacuum
surface = ase.build.surface(gaas, miller, n_GaAs, d_vacuum, periodic=true);

Write an image of the surface and embed it as a nice illustration:

In [4]:
ase.io.write("surface.png", surface * pytuple((3, 3, 1)), rotation="-90x, 30y, -75z")

Python: None

<img src="https://docs.dftk.org/stable/surface.png" width=500 height=500 />

Use the `pyconvert` function from `PythonCall` to convert the ASE atoms
to an AtomsBase-compatible system.
This can then be used in the same way as other `AtomsBase` systems
(see AtomsBase integration for details) to construct a DFTK model:

In [5]:
using DFTK

pseudopotentials = Dict(:Ga => "hgh/pbe/ga-q3.hgh",
                        :As => "hgh/pbe/as-q5.hgh")
model = model_DFT(pyconvert(AbstractSystem, surface);
                  functionals=PBE(),
                  temperature=1e-3,
                  smearing=DFTK.Smearing.Gaussian(),
                  pseudopotentials)

Model(gga_x_pbe+gga_c_pbe, 3D):
    lattice (in Bohr)    : [7.55469   , 0         , 0         ]
                           [0         , 7.55469   , 0         ]
                           [0         , 0         , 40.0648   ]
    unit cell volume     : 2286.6 Bohr³

    atoms                : As₂Ga₂
    atom potentials      : ElementPsp(Ga, "hgh/pbe/ga-q3.hgh")
                           ElementPsp(As, "hgh/pbe/as-q5.hgh")
                           ElementPsp(Ga, "hgh/pbe/ga-q3.hgh")
                           ElementPsp(As, "hgh/pbe/as-q5.hgh")

    num. electrons       : 16
    spin polarization    : none
    temperature          : 0.001 Ha
    smearing             : DFTK.Smearing.Gaussian()

    terms                : Kinetic()
                           AtomicLocal()
                           AtomicNonlocal()
                           Ewald(nothing)
                           PspCorrection()
                           Hartree()
                           Xc(gga_x_pbe, gga_c_pbe)
 

In the above we use the `pseudopotential` keyword argument to
assign the respective pseudopotentials to the imported `model.atoms`.
Try lowering the SCF convergence tolerance (`tol`)
or try `mixing=KerkerMixing()` to see the full challenge of this system.

In [6]:
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis; tol=1e-6, mixing=LdosMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.58678087320                   -0.58    5.2    1.13s
  2   -16.72501532078       -0.86       -1.01    1.0    271ms
  3   -16.73052503273       -2.26       -1.57    2.2    271ms
  4   -16.73121737903       -3.16       -2.16    1.0    256ms
  5   -16.73132162722       -3.98       -2.58    2.1    249ms
  6   -16.73133191254       -4.99       -2.84    1.9    245ms
  7   -16.73082662246   +   -3.30       -2.46    2.1    286ms
  8   -16.73125410646       -3.37       -2.75    2.4    265ms
  9   -16.73113392149   +   -3.92       -2.64    2.0    563ms
 10   -16.73133942626       -3.69       -3.63    1.2    209ms
 11   -16.73133531377   +   -5.39       -3.43    3.1    295ms
 12   -16.73134010316       -5.32       -3.97    1.6    221ms
 13   -16.73133977440   +   -6.48       -3.92    1.3    213ms
 14   -16.73134017018       -6.40       -4.47    1.6    241ms
 15   -16

In [7]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             5.8593983 
    AtomicLocal         -105.6100262
    AtomicNonlocal      2.3494816 
    Ewald               35.5044300
    PspCorrection       0.2016043 
    Hartree             49.5614394
    Xc                  -4.5976641
    Entropy             -0.0000035

    total               -16.731340200429